In [1]:
# == Download the repo content and install dependencies ==
!git clone https://github.com/ifzhang/ByteTrack.git
%cd /content/ByteTrack/
%mkdir pretrained
%cd pretrained

from google.colab import drive 
drive.mount ('/content/drive')

# == Download pretrained X model weights ==
# !gdown --id "1P4mY0Yyd3PPTybgZkjMYhFri88nTmJX5"

Cloning into 'ByteTrack'...
remote: Enumerating objects: 1926, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 1926 (delta 0), reused 0 (delta 0), pack-reused 1921
Receiving objects: 100% (1926/1926), 78.16 MiB | 12.23 MiB/s, done.
Resolving deltas: 100% (1091/1091), done.
/content/ByteTrack
/content/ByteTrack/pretrained
Mounted at /content/drive


In [2]:
!cp /content/drive/MyDrive/project/bytetrack_x_mot17.pth.tar /content/ByteTrack/pretrained

In [3]:
# == Install dependencies ==
!pip3 install cython
!pip3 install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!pip3 install cython_bbox

%cd /content/ByteTrack/
!pip3 install -r requirements.txt

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-hfvbq4bp
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-hfvbq4bp
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.whl size=264361 sha256=da036ceb18db3aa7f3906bcc027cca91505330d7a215b5769c8f436a5641bce3
  Stored in directory: /tmp/pip-ephem-wheel-cache-997vw1am/wheels/e2/6b/1d/344ac773c7495ea0b85eb228bc66daec7400a143a92d36b7b1
Successfully built pycocotools
  Attempting uninstall: pycocotools
    Found existing installation: pycocotools 2.0.4
    Uninstalling pycocotools-2.0.4:
      Successfully uninstalled pycocotools-2.0.4
     |████████████████████████████████| 41 kB 194 kB/s 
  Created wheel for cython-bbox: filename=cython_bbox-0.1.3-cp37-cp37m-linux_x86_64.whl size=57537 sha256=f72bc4fe7014e1370da7901a4dde726befa72c6b99853e11eba54012bc2e0008
  Stored in directory: /root/.cache/pip/wheels/51/82/21/5def8bc98ae4ea436d7f0decb7194d20

In [4]:
# == Install ByteTrack ==
!python3 setup.py develop

running develop
running egg_info
creating yolox.egg-info
writing yolox.egg-info/PKG-INFO
writing dependency_links to yolox.egg-info/dependency_links.txt
writing top-level names to yolox.egg-info/top_level.txt
writing manifest file 'yolox.egg-info/SOURCES.txt'
package init file 'datasets/__init__.py' not found (or not a regular file)
package init file 'deploy/__init__.py' not found (or not a regular file)
package init file 'exps/__init__.py' not found (or not a regular file)
package init file 'assets/__init__.py' not found (or not a regular file)
package init file 'videos/__init__.py' not found (or not a regular file)
package init file 'tools/__init__.py' not found (or not a regular file)
package init file 'pretrained/__init__.py' not found (or not a regular file)
package init file 'tutorials/__init__.py' not found (or not a regular file)
package init file 'yolox/tracker/__init__.py' not found (or not a regular file)
package init file 'yolox/sort_tracker/__init__.py' not found (or not a

In [5]:
# run inference demo (can be slow on colab). The cell output is deflected to the file 'log.txt' for downstream use to display the result but feel free to remove 
%cd /content/ByteTrack
!python3 tools/demo_track.py video -f exps/example/mot/yolox_x_mix_det.py -c pretrained/bytetrack_x_mot17.pth.tar --fp16 --fuse --save_result &> log.txt

/content/ByteTrack


In [6]:
# == Get rendered result video file path ==
import re
%cd /content/ByteTrack
with open('log.txt', 'r') as file:
    text = file.read().replace('\n', '')

m = re.search('video save_path is ./(.+?).mp4', text)
if m:
    found = '/content/ByteTrack/' + m.group(1) + ".mp4"

found

/content/ByteTrack


'/content/ByteTrack/YOLOX_outputs/yolox_x_mix_det/track_vis/2022_04_03_09_34_02/palace.mp4'

In [7]:
# == Convert mp4 to webm video file for proper display on colab ==
%cd /content/ByteTrack
!ffmpeg -i $found -vcodec vp9 out.webm

/content/ByteTrack
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-l

In [8]:
# == Display results ==
from IPython.display import HTML
from base64 import b64encode

video_path = '/content/ByteTrack/out.webm'

mp4 = open(video_path, "rb").read()
data_url = "data:video/webm;base64," + b64encode(mp4).decode()
HTML(f"""
<video width=600 controls>
      <source src="{data_url}" type="video/webm">
</video>
""")